In [7]:
!pip install -U -q -r ../requirements.txt

In [8]:
import os, sys
from dotenv import load_dotenv

In [9]:
rpath = os.path.abspath('/home/user/Documents/10/w7/PromptBuilder for RAG/backend')

if rpath not in sys.path:
    sys.path.insert(0, rpath)

In [10]:
from service.file_processor import FileProcessor
from service.chroma_db_manager import ChromaDBManager
from service.rag_processor import RAGProcessor

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from langchain_openai import ChatOpenAI

In [13]:
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"),model_name="gpt-3.5-turbo", temperature=0)

In [22]:
filePath = os.getenv("PATH_TO_TXT")

# Load and split the pdf file

In [25]:
file = FileProcessor(filePath)
file.load_file()
sp = file.split_file()
sp[30]

Document(page_content='Secondhand cars left us hitchhiking on the side of the freeway. A secondhand car was also the reason my mom got married. If it hadn’t been for the Volkswagen that didn’t work, we never would have looked for the mechanic who became the husband who became the stepfather who became the man who tortured us for years and put a bullet in the back of my mother’s head—I’ll take the new car with the warranty every time.', metadata={'source': '../data/data.txt'})

# Create and store it to chroma_db

In [34]:
chroma = ChromaDBManager()
chromaDb= chroma.store_and_load_chroma_db(sp)
chromaDb

In [38]:
chroma.search_chroma_db_with_score("grandmother")

[(Document(page_content='My great-grandmother lived with us as well. We called her Koko. She was super old, well into her nineties, stooped and frail, completely blind. Her eyes had gone white, clouded over by cataracts. She couldn’t walk without someone holding her up. She’d sit in the kitchen next to the coal stove, bundled up in long skirts and head scarves, blankets over her', metadata={'source': '../data/data.txt'}),
  0.2524379789829254),
 (Document(page_content='My great-grandmother lived with us as well. We called her Koko. She was super old, well into her nineties, stooped and frail, completely blind. Her eyes had gone white, clouded over by cataracts. She couldn’t walk without someone holding her up. She’d sit in the kitchen next to the coal stove, bundled up in long skirts and head scarves, blankets over her', metadata={'source': '../data/data.txt'}),
  0.2524379789829254),
 (Document(page_content='My great-grandmother lived with us as well. We called her Koko. She was super

# RAG

In [40]:
rag_processor = RAGProcessor(llm)
rag_processor.create_rag(chromaDb)
      

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7c6f939581d0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.5})

In [42]:
# create the chain with the templet 
chain = rag_processor.process_rag_chain() 
chain

{
  context: RunnableLambda(itemgetter('question'))
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7c6f939581d0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.5}),
  question: RunnableLambda(itemgetter('question'))
}
| RunnableAssign(mapper={
    context: RunnableLambda(itemgetter('context'))
  })
| {
    response: ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="Create a question based only on the following context, please respond with 'I don't know':\n            {question}\n            Context:\n            {context}\n            "))])
              | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7c6f8ed5cfd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7c6f8ed64850>, temperature=0.0

In [48]:
prompt = "tell me what you know about Koko"

reault= rag_processor.invoker(prompt)
reault["response"].content

"What kind of conversations did Koko have with the narrator's gran?"